## Preprocessing

In [26]:
from openfe import OpenFE, tree_to_formula, transform, TwoStageFeatureSelector
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.metrics import accuracy_score


In [27]:
df = pd.read_csv('F:/Kỳ 4/Thầy Hùng/OpenFE/data/MQTT-IoT-IDS2020/mqtt-iot-ids2020.csv')

In [28]:
df.head(5)

,Unnamed: 0,ip_src,ip_dst,prt_src,prt_dst,proto,fwd_num_pkts,bwd_num_pkts,fwd_mean_iat,bwd_mean_iat,...,fwd_num_bytes,bwd_num_bytes,fwd_num_psh_flags,bwd_num_psh_flags,fwd_num_rst_flags,bwd_num_rst_flags,fwd_num_urg_flags,bwd_num_urg_flags,is_attack,scenario
0,0,192.168.2.5,192.168.2.1,46279,53,17,2,2,0.000008,0.000016,...,166,166,0,0,0,0,0,0,1,mqtt_bruteforce
1,1,192.168.2.7,192.168.2.1,57171,53,17,1,1,0.000000,0.000000,...,87,76,0,0,0,0,0,0,0,mqtt_bruteforce
2,2,192.168.2.7,192.168.2.1,33953,53,17,1,1,0.000000,0.000000,...,87,76,0,0,0,0,0,0,0,mqtt_bruteforce
3,3,192.168.2.6,192.168.2.1,37765,53,17,2,2,0.000007,0.000020,...,134,134,0,0,0,0,0,0,0,mqtt_bruteforce
4,4,192.168.2.7,192.168.2.1,49931,53,17,1,1,0.000000,0.000000,...,71,60,0,0,0,0,0,0,0,mqtt_bruteforce


In [29]:
df.shape

(259379, 34)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259379 entries, 0 to 259378
Data columns (total 34 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         259379 non-null  int64  
 1   ip_src             259379 non-null  object 
 2   ip_dst             259379 non-null  object 
 3   prt_src            259379 non-null  int64  
 4   prt_dst            259379 non-null  int64  
 5   proto              259379 non-null  int64  
 6   fwd_num_pkts       259379 non-null  int64  
 7   bwd_num_pkts       259379 non-null  int64  
 8   fwd_mean_iat       259379 non-null  float64
 9   bwd_mean_iat       259379 non-null  float64
 10  fwd_std_iat        259379 non-null  float64
 11  bwd_std_iat        259379 non-null  float64
 12  fwd_min_iat        259379 non-null  float64
 13  bwd_min_iat        259379 non-null  float64
 14  fwd_max_iat        259379 non-null  float64
 15  bwd_max_iat        259379 non-null  float64
 16  fw

In [31]:
df.isnull().sum()

Unnamed: 0           0
ip_src               0
ip_dst               0
prt_src              0
prt_dst              0
proto                0
fwd_num_pkts         0
bwd_num_pkts         0
fwd_mean_iat         0
bwd_mean_iat         0
fwd_std_iat          0
bwd_std_iat          0
fwd_min_iat          0
bwd_min_iat          0
fwd_max_iat          0
bwd_max_iat          0
fwd_mean_pkt_len     0
bwd_mean_pkt_len     0
fwd_std_pkt_len      0
bwd_std_pkt_len      0
fwd_min_pkt_len      0
bwd_min_pkt_len      0
fwd_max_pkt_len      0
bwd_max_pkt_len      0
fwd_num_bytes        0
bwd_num_bytes        0
fwd_num_psh_flags    0
bwd_num_psh_flags    0
fwd_num_rst_flags    0
bwd_num_rst_flags    0
fwd_num_urg_flags    0
bwd_num_urg_flags    0
is_attack            0
scenario             0
dtype: int64

In [32]:
df[df.duplicated()]

,Unnamed: 0,ip_src,ip_dst,prt_src,prt_dst,proto,fwd_num_pkts,bwd_num_pkts,fwd_mean_iat,bwd_mean_iat,...,fwd_num_bytes,bwd_num_bytes,fwd_num_psh_flags,bwd_num_psh_flags,fwd_num_rst_flags,bwd_num_rst_flags,fwd_num_urg_flags,bwd_num_urg_flags,is_attack,scenario


In [33]:
df['is_attack'].unique()

array([1, 0], dtype=int64)

In [34]:
value_counts = df['scenario'].value_counts()
value_counts

scenario
sparta             91318
normal             86008
scan_sU            39664
scan_A             25693
mqtt_bruteforce    16696
Name: count, dtype: int64

In [35]:
df.select_dtypes(include=['object']).columns

Index(['ip_src', 'ip_dst', 'scenario'], dtype='object')

In [36]:
#Encode
label_encoder = LabelEncoder()
cols_to_encode = ['scenario']
for col in cols_to_encode:
    if df[col].dtype == 'object':
        df[col] = label_encoder.fit_transform(df[col])

In [37]:
for col in df.columns:
  if len(df[col].unique()) <= 1:
    print(col)
    df.drop(col, axis=1, inplace=True)

bwd_num_urg_flags


In [38]:
features = df.drop(['is_attack', 'ip_src', 'ip_dst', 'Unnamed: 0', 'bwd_num_psh_flags'], axis=1)
labels = df[['is_attack']]

In [39]:
features.head()

,prt_src,prt_dst,proto,fwd_num_pkts,bwd_num_pkts,fwd_mean_iat,bwd_mean_iat,fwd_std_iat,bwd_std_iat,fwd_min_iat,...,bwd_min_pkt_len,fwd_max_pkt_len,bwd_max_pkt_len,fwd_num_bytes,bwd_num_bytes,fwd_num_psh_flags,fwd_num_rst_flags,bwd_num_rst_flags,fwd_num_urg_flags,scenario
0,46279,53,17,2,2,0.000008,0.000016,0.0,0.0,0.000008,...,83.0,83.0,83.0,166,166,0,0,0,0,0
1,57171,53,17,1,1,0.000000,0.000000,0.0,0.0,0.000000,...,76.0,87.0,76.0,87,76,0,0,0,0,0
2,33953,53,17,1,1,0.000000,0.000000,0.0,0.0,0.000000,...,76.0,87.0,76.0,87,76,0,0,0,0,0
3,37765,53,17,2,2,0.000007,0.000020,0.0,0.0,0.000007,...,67.0,67.0,67.0,134,134,0,0,0,0,0
4,49931,53,17,1,1,0.000000,0.000000,0.0,0.0,0.000000,...,60.0,71.0,60.0,71,60,0,0,0,0,0


## Training

In [40]:
X, _, y, _ = train_test_split(features, labels, train_size=50000, stratify=labels, random_state=42)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Generate new feature by OpenFE

In [42]:
ofe = OpenFE()
features = ofe.fit(data = X_train, label = y_train) 

The number of candidate features is 4302
Start stage I selection.


100%|██████████| 4/4 [02:31<00:00, 37.77s/it]


2313 same features have been deleted.
Meet early-stopping in successive feature-wise halving.


100%|██████████| 4/4 [03:06<00:00, 46.74s/it]


The number of remaining candidate features is 1989
Start stage II selection.


100%|██████████| 4/4 [00:39<00:00,  9.84s/it]


Finish data processing.
[LightGBM] [Info] Number of positive: 1649, number of negative: 4351
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.269269 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 344843
[LightGBM] [Info] Number of data points in the train set: 6000, number of used features: 2014
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

In [43]:
# Set hyperparameters for binary classification
params = {
    'objective': 'binary',  # Đặt mục tiêu là phân loại nhị phân
    'metric': 'binary_error',  # Đánh giá bằng binary log loss
    'boosting_type': 'gbdt',
    'seed' : 1
}

## Score function

In [44]:
def get_score(train_x, test_x, train_y, test_y):
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
    gbm = lgb.LGBMClassifier(**params)
    gbm.fit(train_x, train_y, eval_set=[(val_x, val_y)])
    pred = gbm.predict(test_x)
    score = accuracy_score(test_y, pred)
    return score


### Before Feature Generation

In [45]:
# get baseline score
score = get_score(X_train, X_test, y_train, y_test)
print("The score before feature generation is", score)

[LightGBM] [Info] Number of positive: 1644, number of negative: 4356
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000900 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4277
[LightGBM] [Info] Number of data points in the train set: 6000, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.274000 -> initscore=-0.974422
[LightGBM] [Info] Start training from score -0.974422
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

### After Feature Generation

In [46]:
X_train, X_test = transform(X_train, X_test, ofe.new_features_list[:10], n_jobs = 1) 

In [47]:
X_train.columns

Index(['prt_src', 'prt_dst', 'proto', 'fwd_num_pkts', 'bwd_num_pkts',
       'fwd_mean_iat', 'bwd_mean_iat', 'fwd_std_iat', 'bwd_std_iat',
       'fwd_min_iat', 'bwd_min_iat', 'fwd_max_iat', 'bwd_max_iat',
       'fwd_mean_pkt_len', 'bwd_mean_pkt_len', 'fwd_std_pkt_len',
       'bwd_std_pkt_len', 'fwd_min_pkt_len', 'bwd_min_pkt_len',
       'fwd_max_pkt_len', 'bwd_max_pkt_len', 'fwd_num_bytes', 'bwd_num_bytes',
       'fwd_num_psh_flags', 'fwd_num_rst_flags', 'bwd_num_rst_flags',
       'fwd_num_urg_flags', 'scenario', 'autoFE_f_0', 'autoFE_f_1',
       'autoFE_f_2', 'autoFE_f_3', 'autoFE_f_4', 'autoFE_f_5', 'autoFE_f_6',
       'autoFE_f_7', 'autoFE_f_8', 'autoFE_f_9'],
      dtype='object')

In [48]:
score = get_score(X_train, X_test, y_train, y_test)

[LightGBM] [Info] Number of positive: 1644, number of negative: 4356
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004428 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6093
[LightGBM] [Info] Number of data points in the train set: 6000, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.274000 -> initscore=-0.974422
[LightGBM] [Info] Start training from score -0.974422
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [49]:
# OpenFE recommends a list of new features. We include the top 10
# generated features to see how they influence the model performance
print("The score after feature generation is", score)
print("The top 10 generated features are")
for feature in ofe.new_features_list[:10]:
    print(tree_to_formula(feature))

The score after feature generation is 0.9984
The top 10 generated features are
(bwd_num_pkts-fwd_mean_pkt_len)
CombineThenFreq(bwd_max_pkt_len,scenario)
GroupByThenNUnique(fwd_max_pkt_len,bwd_min_pkt_len)
GroupByThenRank(prt_dst,bwd_max_pkt_len)
(bwd_std_iat-bwd_num_rst_flags)
(fwd_mean_iat/bwd_min_iat)
GroupByThenStd(fwd_mean_pkt_len,bwd_max_pkt_len)
(prt_dst-scenario)
GroupByThenRank(prt_dst,fwd_min_pkt_len)
GroupByThenRank(prt_src,scenario)
